In [ ]:
import json
from pathlib import Path
import sys
import os
import ast
import re
sys.path.append(os.path.abspath('..'))

import pandas as pd

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

BASE_DIR = Path.cwd().parent
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
company = '3m'
year = 2022
blsheet_path = BASE_DIR / f'data/extracted_balance_sheet/{company}_{year}.json'
asset_category_reference_path = BASE_DIR / 'data/reference/asset_category_reference.txt'
query_path = BASE_DIR / 'data/query/eng_query.csv'

with open(blsheet_path) as f:
    blsheet = json.load(f)
with open(asset_category_reference_path) as f:  
    category_description = f.read()

# 1. Queryの準備

In [ ]:
# queryの準備
query_df = pd.read_csv(query_path).rename(columns={'Item': 'Query'})

# 大分類ごとのクエリを作成
current_asset_query_df = query_df.loc[0:10]
current_asset_markdown_query = current_asset_query_df.to_markdown(index=False)

fixed_asset_query_df = query_df.loc[11:24]
fixed_asset_markdown_query = fixed_asset_query_df.to_markdown(index=False)

deferred_charges_query_df = query_df.loc[25:25]
deffered_charges_markdown_query = deferred_charges_query_df.to_markdown(index=False)

total_asset_raw = query_df.loc[26:26]

# 2. total assetを取得

In [ ]:
def pop_total_asset_value(blsheet: list):
    total_amount = 0
    for category in blsheet:
        if category['name'].lower() == 'total assets':
            total_asset_category = category
            blsheet.remove(category)
            return total_asset_category['accounts'][0]['amount']
        else:
            total_amount += sum(
                account['amount'] for account in category['accounts'] if 'total' not in account['name'].lower()
            )
    return total_amount

total_asset_raw = total_asset_raw.copy()
total_asset_raw.loc[:, 'prediction'] = pop_total_asset_value(blsheet)

# 2. 出力フォーマットに従って残りの項目を回答

In [ ]:
# gpt4-oのoutputをint型に矯正
def validate_output(output):
    numbers = re.findall(r'-?\d+', output)
    if numbers:
        return int(numbers[-1])
    else:
        return output

# 抽出結果の評価関数
def evaluate_prediction(result_df):
    result_df['prediction'] = result_df['prediction'].astype(int)
    result_df['judgement'] = result_df['prediction'] == result_df['Answer (M USD)']
    num_correect = len(result_df[result_df['judgement']])
    print(f'number of correct answers: {num_correect}, acduracy rate: {num_correect / len(result_df)}')     
    return result_df

## 実験: 参照データが 全範囲 or 大分類ごとで精度比較

### 1. 全範囲 -> 正答率 **46%** (12/26)

In [ ]:
# 参照データの作成
simple_asset_items = {'Item_name': [], 'Amount': []}
for category in blsheet:
    for account in category['accounts']:
        simple_asset_items['Item_name'].append(account['name'])
        simple_asset_items['Amount'].append(account['amount'])
simple_asset_items_df = pd.DataFrame(simple_asset_items)
simple_asset_items_markdown = simple_asset_items_df.to_markdown(index=False)

In [ ]:
def retrieve_asset_amount(raw):
    template = (
        "[Instruction]\n"
        "Your task is identifying and calculating the total amount for a query item from the provided balance sheet data.\n\n"
        "Step to follow:\n"
        "1. Search for an exact match for the query item in the 'Item_name' column of the balance sheet.\n"
        "2. If no exact match is found, interpret the 'Description' and search for items with similar meanings or expressions, "
        "even if the wording is different. Consider combinations of multiple rows that could sum up to represent the query item\n"
        "3. If, after these steps, no corresponding item can be identified, output a value of 0\n\n"
        "[balance sheet]\n{markdown_assets_items}\n\n"
        "[query]\n - Item name: {query}\n - Description: {desc}\n\n"
        "Output:\n Only provide the total amount as a single integer value such as '-200'."
        "Do not include any explanations, calculations and other currency symbols ($).\n\n"
    )
    massage_prompt = PromptTemplate(input_variable=["markdown_assets_items", "markdown_query"], template=template)
    prompt = massage_prompt.format_prompt(markdown_assets_items=simple_asset_items_markdown, query=raw['Query'], desc=raw['Description'])
    client = ChatOpenAI(temperature=0.8)
    return validate_output(client.invoke(prompt).content)

query_df['prediction'] = query_df.apply(retrieve_asset_amount, axis=1)
reslt_df = evaluate_prediction(query_df)
print(reslt_df[['Query', 'prediction', 'Answer (M USD)', 'judgement']].to_markdown(index=False))

### 3. 大分類ごと　--> 正答率: 54% (14/26)

In [ ]:
categories = [category['name'] for category in blsheet]
template = (
    "[Setting]\n"
    "You are analyzing a balance sheet and need to classify its categories into one of the following three groups:"
    f"[Category Description]\n{category_description}\n\n"
    "[Instruction]\n"
    "Analyze the provided **[list of balance sheet] items."
    "For each item, determine whether it falls under three group based on the descriptions above"
    "[format]\n"
    "dict['Current assets': [], 'Fixed assets': [], 'Deferred charges': [], 'Others': []]\n\n"
    "It is acceptable if there is a group with no corresponding categories\n\n"
    f"**[list of balance sheet items]\n{categories}"
    ""
)
massage_prompt = PromptTemplate(input_variable=["categories"], template=template)
prompt = massage_prompt.format_prompt(categories=categories)
client = ChatOpenAI(temperature=0.0)

clasified_categories = client.invoke(prompt).content
clasified_categoriy_dict = ast.literal_eval(clasified_categories)
print(clasified_categories)

In [ ]:
current_assets_items = []
fixed_assets_items = []
deferred_charges_items = []
other_items = []

for category in blsheet:
    if category['name'] in clasified_categoriy_dict['Current assets']:
        current_assets_items.extend(category['accounts'])
    elif category['name'] in clasified_categoriy_dict['Fixed assets']:
        fixed_assets_items.extend(category['accounts'])
    elif category['name'] in clasified_categoriy_dict['Deferred charges']:
        deferred_charges_items.extend(category['accounts'])
    elif category['name'] in clasified_categoriy_dict['Others']:
        other_items.extend(category['accounts'])

current_assets_item_df = pd.DataFrame(current_assets_items).rename(columns={'name': 'Item_name', 'amount': 'Amount'})
fixed_assets_item_df = pd.DataFrame(fixed_assets_items).rename(columns={'name': 'Item_name', 'amount': 'Amount'})
deferred_charges_item_df = pd.DataFrame(deferred_charges_items).rename(columns={'name': 'Item_name', 'amount': 'Amount'})
other_item_df = pd.DataFrame(other_items).rename(columns={'name': 'Item_name', 'amount': 'Amount'})

markdown_current_assets_items = current_assets_item_df.to_markdown(index=False)
markdown_fixed_assets_items = fixed_assets_item_df.to_markdown(index=False)
markdown_deferred_charges_items = deferred_charges_item_df.to_markdown(index=False)
markdown_other_items = other_item_df.to_markdown(index=False)

In [ ]:
def retrieve_asset_amount(raw, markdown_assets_items):
    template = (
        "[Instruction]\n"
        "Your task is identifying and calculating the total amount for a query item from the provided balance sheet data.\n\n"
        "Step to follow:\n"
        "1. Search for an exact match for the query item in the 'Item_name' column of the balance sheet.\n"
        "2. If no exact match is found, search for item name with similar meanings or expressions with interpreting the 'Description'"
        "3. If no relevant items are found in the balance sheet, refer to the [Other Assets] table and repeat steps 1 and 2\n"
        "even if the wording is different. Consider combinations of multiple rows that could sum up to represent the query item\n"
        "4. If, after these steps, no corresponding item can be identified, output a value of 0\n\n"
        "[balance sheet]\n{markdown_assets_items}\n\n"
        "[other assets]\n{markdown_other_items}\n\n"
        "[query]\n - Item name: {query}\n - Description: {desc}\n\n"
        "Output:\n Only provide the total amount as a single integer value such as '-200'."
        "Do not include any explanations, calculations and other currency symbols.\n\n"
    )
    massage_prompt = PromptTemplate(input_variable=["markdown_assets_items", "markdown_query"], template=template)
    prompt = massage_prompt.format_prompt(
        markdown_assets_items=markdown_assets_items, markdown_other_items=markdown_other_items, query=raw['Query'], desc=raw['Description'])
    client = ChatOpenAI(temperature=0.0)
    response = client.invoke(prompt).content
    return validate_output(response)

current_asset_query_df.loc[:, 'prediction'] = current_asset_query_df.apply(lambda x: retrieve_asset_amount(x, markdown_current_assets_items), axis=1)
fixed_asset_query_df.loc[:, 'prediction'] = fixed_asset_query_df.apply(lambda x: retrieve_asset_amount(x, markdown_fixed_assets_items), axis=1)
deferred_charges_query_df.loc[:, 'prediction'] = deferred_charges_query_df.apply(lambda x: retrieve_asset_amount(x, markdown_deferred_charges_items), axis=1)
result_df = pd.concat([current_asset_query_df, fixed_asset_query_df, deferred_charges_query_df], axis=0)
evaluate_prediction(result_df)
print(reslt_df[['Query', 'prediction', 'Answer (M USD)', 'judgement']].to_markdown(index=False))

# 3: total assetsと結合して最終成果物作成

In [ ]:
submit_df = pd.concat([total_asset_raw, result_df], axis=0)
submit_df = evaluate_prediction(submit_df)
print(submit_df[['Query', 'prediction', 'Answer (M USD)', 'judgement']].to_markdown(index=False))